
# Helper Functions

This notebook includes helper functions that are imported by other notebooks to perform certain tasks.


In [4]:
import numpy as np
import matplotlib as plt
from sklearn.metrics import mean_absolute_percentage_error


## PredictAndForecast Class Overview

The `PredictAndForecast` class leverages a trained machine learning model to generate predictions. It is tailored for time series forecasting, where predictions are based on a sequence of data from both training and testing datasets. The class facilitates the simulation of real-time predictions, reflecting how a model might perform in an operational setting.

### Key Functionalities:
- **Dynamic Forecasting**: Utilizes historical data to predict future values step-by-step, mimicking real-world forecasting tasks.
- **Seamless Integration**: Designed to work smoothly with any trained `tf.keras.Model`, providing flexibility in model choice.

### Operational Method:
- **Forecast Method**: Projects future values using the model based on the latest available data.
- **Prediction Compilation**: Systematically generates and compiles predictions across the test set, building a complete forecast series.


In [ ]:
class PredictAndForecast:
    """
    model: tf.keras.Model
    train: np.array
    test: np.array
    Takes a trained model, train, and test datasets and returns predictions
    of len(test) with same shape.
    """
    def __init__(self, model, train, test, n_input=5) -> None:
        self.model = model
        self.train = train
        self.test = test
        self.n_input = n_input
        self.predictions = self.get_predictions()

    def forecast(self, history) -> np.array:
        """
        Given last weeks actual data, forecasts next weeks prices.
        """
        # flatten data
        data = np.array(history)
        data = data.reshape((data.shape[0]*data.shape[1], data.shape[2]))
        # retrieve last observations for input data
        input_x = data[-self.n_input:, :]
        # reshape into [1, n_input, 1]
        input_x = input_x.reshape((1, len(input_x), input_x.shape[1]))
        # forecast the next week
        yhat = self.model.predict(input_x, verbose=0)
        # we only want the vector forecast
        yhat = yhat[0]
        return yhat

    def get_predictions(self) -> np.array:
        """
        compiles models predictions week by week over entire
        test set.
        """
        # history is a list of weekly data
        history = [x for x in self.train]
        # walk-forward validation over each week
        predictions = []
        for i in range(len(self.test)):
            yhat_sequence = self.forecast(history)
            # store the predictions
            predictions.append(yhat_sequence)
        # get real observation and add to history for predicting the next week
            history.append(self.test[i, :])
        return np.array(predictions)

## Evaluate Class Overview

The `Evaluate` class is designed to quantify the performance of forecasting models using standard statistical metrics. It provides a straightforward approach to measure how closely predictions align with actual outcomes, offering insights into model accuracy and reliability.

### Key Functionalities:
- **Accuracy Assessment**: Evaluates model predictions using metrics like variance ratio and Mean Absolute Percentage Error (MAPE).
- **Model Performance Insight**: Delivers critical data on the effectiveness of the model in capturing and replicating the underlying data patterns.

### Operational Methods:
- **Variance Ratio Calculation**: Assesses how well the model predictions capture the variability of the actual data.
- **MAPE Calculation**: Measures the average percentage error between the model's predictions and the actual data, offering a clear view of model precision.


In [ ]:
class Evaluate:

  def __init__(self, actual, predictions) -> None:
    self.actual = actual
    self.predictions = predictions
    self.var_ratio = self.compare_var()
    self.mape = self.evaluate_model_with_mape()

  def compare_var(self):
    return abs( 1 - (np.var(self.predictions) / np.var(self.actual)))

  def evaluate_model_with_mape(self):
    return mean_absolute_percentage_error(self.actual.flatten(), self.predictions.flatten())

## Visualization Function

This function visualizes the actual and predicted values of the timeseries over time, to provide a clear comparison between the two. It is used to evaluate the performance of the forecasting model visually.

In [5]:
def plot_results(test, preds, df, image_path=None, title_suffix=None, xlabel='AAPL stock Price'):
  """
  Plots training data in blue, actual values in red, and predictions in green,
  over time.
  """
  fig, ax = plt.subplots(figsize=(20,6))
  # x = df.Close[-498:].index
  plot_test = test[1:]
  plot_preds = preds[1:]
  # plot_test = np.array(plot_test)

  x = df[-(plot_test.shape[0]*plot_test.shape[1]):].index.to_numpy()
  plot_test = plot_test.reshape((plot_test.shape[0]*plot_test.shape[1], 1))
  plot_preds = plot_preds.reshape((plot_test.shape[0]*plot_test.shape[1], 1))
  ax.plot(x, plot_test, label='actual')
  ax.plot(x, plot_preds, label='preds')
  if title_suffix==None:
    ax.set_title('Predictions vs. Actual')
  else:
    ax.set_title(f'Predictions vs. Actual, {title_suffix}')
  ax.set_xlabel('Date')
  ax.set_ylabel(xlabel)
  ax.legend()
  plt.show()